# Pre-processing Wyoming Water Right data for WaDEQA upload.
Purpose:  To pre-process the Wyoming data into one master file for simple DataFrame creation and extraction

Notes:
- Date Updated: 05/06/2022
- Merging GW and SW sources into one workable input.

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Wyoming/WaterAllocation/RawInputData"
os.chdir(workingDir)

# POD Data

### POD Groundwater

In [3]:
# Input File
GW_Input = "POD_GW_DepthI_FC_input.zip"
dfwyg = pd.read_csv(GW_Input).replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfwyg:
    dfwyg['WaDEUUID'] = "wyDG" + dfwyg.index.astype(str)
    dfwyg.to_csv('POD_GW_DepthI_FC_input.zip', compression=dict(method='zip', archive_name='POD_GW_DepthI_FC_input.csv'), index=False)

print(len(dfwyg))
dfwyg.head()

2323


,WaDEUUID,ReasonRemoved,IncompleteField,OID_,IsActive,SummaryCRStatus,WRPODID,WR_Number,PriorityDate,PriorityText,SummaryWRStatus,Company,FirstName,LastName,FacilityName,Uses,Twn,Rng,Sec,Qtr_Qtr,Survey_Type_Survey_Number_Survey_Suffix,Total_Flow_CFS___Appropriation_GPM_,Total_depth__Ft_,StaticWaterLevel__Ft_,Well_Log_Y_N_,Total_Capacity__AF_Yr_,Facility_type,SupplyType,Chemical_Analysis_Y_N_,CreatedBy,QTRQTR_CLEAN,DateDownloaded,WaterDivision,WaterDistrict,Longitude_Double,Latitude_Double,Contractor,Contract_ID,HORIZONTAL_ACCURACY,Creation_Edit_Date,PermitNum,OrdrRecNum,PermitPrefix,PermitSuffix,Pump_Depth,PWBFDepTop,PWBFDepBot
0,wyDG0,Unused Site Record,,1,A,A,,CR UW04/059,8/5/1976,8/5/1976,,,INC.,O. W. RANCH,O W WELL #1,IRR_GW,034N,063W,19,NE1/4NE1/4,,575.00000,,,,,Well,Original,,External,NENE,3/2/2019 0:00,2.00000,2.00000,-104.46127,42.91290,CCI,05SC0297516,QTR2,4/10/2020 0:00,34894,OR 25/527,P,W,,,
1,wyDG1,,,2,A,,,P210075.0W,11/28/2018,,Incomplete,FREESTONE MIDSTREAM,,,LINCOLN WATER WELL #1,MIS,034N,069W,26,NW1/4NW1/4,A,750.00000,,,,,Well,,,External,NWNW,3/2/2019 0:00,2.00000,2.00000,-105.11490,42.89665,CCI,05SC0297516,QTR2,4/10/2020 0:00,210075,,P,W,,,
2,wyDG10,Unused Site Record,,11,A,,,P114751.0W,3/31/1999,,Incomplete,"Power Resources, Inc.",,,NE/NW 13-36-73; (59 WELLS),IND_GW,036N,073W,13,NE1/4NW1/4,A5-,885.00000,600.0,335.0,,,Well,,N,External,NENW,3/2/2019 0:00,2.00000,2.00000,-105.56776,43.09796,CCI,05SC0297516,QTR2,4/10/2020 0:00,114751,,P,W,,-1,-1
3,wyDG100,,,101,A,,,P205103.0W,2/8/2016,,Incomplete,PEABODY SCHOOL CREEK MINING LLC,,,SCM-EAGLE-1,MIS,041N,070W,6,SE1/4SW1/4,L21,4000.00000,,,,,Well,,,External,SESW,3/2/2019 0:00,2.00000,2.00000,-105.31532,43.55047,CCI,05SC0297516,QTR2,4/10/2020 0:00,205103,,P,W,,,
4,wyDG1000,Unused Site Record,,1001,A,A,,CR UW05/085,5/20/1976,5/20/1976,Fully Adjudicated,,HENRY AND EDNA,SCHNEIDER,SCHNEIDER NO 6 WELL,IRR_GW,029N,061W,29,SW1/4SE1/4,A,700.00000,,,,,Well,Supplemental,,External,SWSE,3/2/2019 0:00,1.00000,1.00000,-104.21279,42.45147,CCI,05SC0297516,QTR2,4/10/2020 0:00,33227,OR 30/242,P,W,,,


In [4]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
dfwyg['PriorityDate'] = pd.to_datetime(dfwyg['PriorityDate'])
dfwyg['PriorityDate'] = pd.to_datetime(dfwyg["PriorityDate"].dt.strftime('%m/%d/%Y'))
dfwyg.head(1)

,WaDEUUID,ReasonRemoved,IncompleteField,OID_,IsActive,SummaryCRStatus,WRPODID,WR_Number,PriorityDate,PriorityText,SummaryWRStatus,Company,FirstName,LastName,FacilityName,Uses,Twn,Rng,Sec,Qtr_Qtr,Survey_Type_Survey_Number_Survey_Suffix,Total_Flow_CFS___Appropriation_GPM_,Total_depth__Ft_,StaticWaterLevel__Ft_,Well_Log_Y_N_,Total_Capacity__AF_Yr_,Facility_type,SupplyType,Chemical_Analysis_Y_N_,CreatedBy,QTRQTR_CLEAN,DateDownloaded,WaterDivision,WaterDistrict,Longitude_Double,Latitude_Double,Contractor,Contract_ID,HORIZONTAL_ACCURACY,Creation_Edit_Date,PermitNum,OrdrRecNum,PermitPrefix,PermitSuffix,Pump_Depth,PWBFDepTop,PWBFDepBot
0,wyDG0,Unused Site Record,,1,A,A,,CR UW04/059,1976-08-05,8/5/1976,,,INC.,O. W. RANCH,O W WELL #1,IRR_GW,034N,063W,19,NE1/4NE1/4,,575.00000,,,,,Well,Original,,External,NENE,3/2/2019 0:00,2.00000,2.00000,-104.46127,42.91290,CCI,05SC0297516,QTR2,4/10/2020 0:00,34894,OR 25/527,P,W,,,


In [5]:
#Creating WaDE Owner Field.  
#Create from Owner field. If empty, use LastName + FirstName fields.

def retrieveOwner(Com, FN, LN):
    Com = str(Com).strip()
    FN = str(FN).strip()
    LN = str(LN).strip()
    if Com == "" or pd.isnull(Com):
        outList = LN + ", " + FN
    else:
        outList = Com
    return outList
dfwyg['WaDEOwner'] = dfwyg.apply(lambda row: retrieveOwner(row['Company'], row['FirstName'], row['LastName']), axis=1)
dfwyg.head(1)

,WaDEUUID,ReasonRemoved,IncompleteField,OID_,IsActive,SummaryCRStatus,WRPODID,WR_Number,PriorityDate,PriorityText,SummaryWRStatus,Company,FirstName,LastName,FacilityName,Uses,Twn,Rng,Sec,Qtr_Qtr,Survey_Type_Survey_Number_Survey_Suffix,Total_Flow_CFS___Appropriation_GPM_,Total_depth__Ft_,StaticWaterLevel__Ft_,Well_Log_Y_N_,Total_Capacity__AF_Yr_,Facility_type,SupplyType,Chemical_Analysis_Y_N_,CreatedBy,QTRQTR_CLEAN,DateDownloaded,WaterDivision,WaterDistrict,Longitude_Double,Latitude_Double,Contractor,Contract_ID,HORIZONTAL_ACCURACY,Creation_Edit_Date,PermitNum,OrdrRecNum,PermitPrefix,PermitSuffix,Pump_Depth,PWBFDepTop,PWBFDepBot,WaDEOwner
0,wyDG0,Unused Site Record,,1,A,A,,CR UW04/059,1976-08-05,8/5/1976,,,INC.,O. W. RANCH,O W WELL #1,IRR_GW,034N,063W,19,NE1/4NE1/4,,575.00000,,,,,Well,Original,,External,NENE,3/2/2019 0:00,2.00000,2.00000,-104.46127,42.91290,CCI,05SC0297516,QTR2,4/10/2020 0:00,34894,OR 25/527,P,W,,,,"O. W. RANCH, INC."


In [6]:
#Creating Beneficial Use.
#Need to translate WY abbreviatoins to a workable format.

BenUseDict = {
"AESCNG" : "Coal Bed Natural Gas",
"AESFIS" : "Fish Propagation (Aesthetics)",
"AESGWR" : "Ground Water Recharge (Aesthetics)",
"AESREC" : "Recreation (Aesthetics)",
"AESSTK" : "Stock (Aesthetics)",
"AESWET" : "Wetlands (Aesthetics)",
"AESWIL" : "Wildlife (Aesthetics)",
"AQU" : "Aquaculture",
"BOT" : "Bottling Water",
"CAG" : "Commercial Agriculture",
"CBM" : "Coal Bed Methane - Ground Water",
"CHE" : "Chemical",
"CIS" : "Consumptive Instream Flow",
"CMU" : "Combined Uses",
"CNG_SW" : "Coal Bed Natural Gas",
"COM" : "Commercial",
"CUL" : "Culinary",
"DAI" : "Dairy",
"DEW" : "Mine Dewatering",
"DOM_GW" : "Domestic - Ground Water",
"DOM_SW" : "Domestic - Surface Water",
"DPA" : "Domestic (Phase 2 Award)",
"DRI" : "Drilling",
"DSP" : "Domestic Supply",
"DTA" : "Dust Abatement",
"ECAP" : "Existing Capacity",
"ERO" : "Erosion Control",
"FIR" : "Fire Protection",
"FIS" : "Fish Propagation",
"FLO" : "Flood Control",
"FTH" : "Flow Through",
"GWR" : "Ground Water Recharge",
"HEX" : "Heat Extraction",
"HWY" : "Highway Construction",
"HYD" : "Hydropower",
"HYT" : "Hydrostatic Testing",
"ICE" : "Ice Cutting",
"IFA" : "Instream Flow (Phase 2 Award)",
"IND_GW" : "Industrial - Ground Water",
"IND_SW" : "Industrial - Surface Water",
"IRR_GW" : "Irrigation - Ground Water",
"IRR_SW" : "Irrigation - Surface Water",
"ISF" : "Instream Flow",
"LAK" : "Maintain Natural Lake Level (Phase 2 Award)",
"LAW" : "Large Scale Landscape",
"MAI" : "Maintenance (Equipment Washing)",
"MAN" : "Manufacturing",
"MEC" : "Mechanical",
"MED" : "Medicinal",
"MEM" : "Municipal (Emergency)",
"MIL" : "Milling",
"MIN" : "Mining",
"MIS" : "Miscellaneous - Ground Water",
"MON" : "Monitor",
"MUN_GW" : "Municipal - Ground Water",
"MUN_SW" : "Municipal - Surface Water",
"NAT" : "Natural Flow (Phase 2 Award)",
"O&G" : "Oil and Gas Well Drilling",
"OIL" : "Oil",
"OTH" : "Other",
"OTH_CM" : "Other - Commercial",
"OTH_IN" : "Other - Industrial",
"OTH_TM" : "Other - Temporary",
"P&S" : "Potable and Sanitary Supply",
"PCT" : "Pollution Control",
"POW" : "Power",
"RAI" : "Railroad",
"RDC" : "Road Construction",
"REC" : "Recreation",
"REF" : "Refining",
"RES" : "Reservoir Supply",
"REW" : "Reclamation Watering",
"S&D" : "Stock and Domestic",
"SDG" : "Gpm For Domestic or Stock",
"SDU" : "Stock and Domestic",
"SED" : "Sediment Control",
"SNO" : "Snow Making",
"STE" : "Stream",
"STK" : "Stock Watering",
"STKNDMS" : "Stock and Domestic",
"STO" : "Stock",
"STS" : "Stock",
"STW" : "Stock Watering",
"SWD" : "Subdivision",
"SWP" : "Stock Water Pipeline",
"TEM" : "Temporary",
"TENL" : "Total Enlargement",
"TRA" : "Transportation",
"TST" : "Test Well",
"TWR" : "Tree Watering",
"UTL" : "Utilities",
"W&S" : "Wild and Scenic",
"WDR" : "Well Drilling",
"WET" : "Wetlands",
"WHL" : "Water Hauls",
"WL" : "Wildlife"}

def retrieveBenUse(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = ""
    else:
        String1 = str(colrowValue).strip()
        x = String1.split(';')
        try:
            outList = []
            for i in range(len(x)):
                y = x[i].strip()
                y = BenUseDict[y]
                outList.append(y)
            outString = ", ".join(str(e) for e in outList)
        except:
            outString = ""
    return outString

dfwyg['WaDEBenUse'] = dfwyg.apply(lambda row: retrieveBenUse(row['Uses']), axis=1)
dfwyg.head()

,WaDEUUID,ReasonRemoved,IncompleteField,OID_,IsActive,SummaryCRStatus,WRPODID,WR_Number,PriorityDate,PriorityText,SummaryWRStatus,Company,FirstName,LastName,FacilityName,Uses,Twn,Rng,Sec,Qtr_Qtr,Survey_Type_Survey_Number_Survey_Suffix,Total_Flow_CFS___Appropriation_GPM_,Total_depth__Ft_,StaticWaterLevel__Ft_,Well_Log_Y_N_,Total_Capacity__AF_Yr_,Facility_type,SupplyType,Chemical_Analysis_Y_N_,CreatedBy,QTRQTR_CLEAN,DateDownloaded,WaterDivision,WaterDistrict,Longitude_Double,Latitude_Double,Contractor,Contract_ID,HORIZONTAL_ACCURACY,Creation_Edit_Date,PermitNum,OrdrRecNum,PermitPrefix,PermitSuffix,Pump_Depth,PWBFDepTop,PWBFDepBot,WaDEOwner,WaDEBenUse
0,wyDG0,Unused Site Record,,1,A,A,,CR UW04/059,1976-08-05,8/5/1976,,,INC.,O. W. RANCH,O W WELL #1,IRR_GW,034N,063W,19,NE1/4NE1/4,,575.00000,,,,,Well,Original,,External,NENE,3/2/2019 0:00,2.00000,2.00000,-104.46127,42.91290,CCI,05SC0297516,QTR2,4/10/2020 0:00,34894,OR 25/527,P,W,,,,"O. W. RANCH, INC.",Irrigation - Ground Water
1,wyDG1,,,2,A,,,P210075.0W,2018-11-28,,Incomplete,FREESTONE MIDSTREAM,,,LINCOLN WATER WELL #1,MIS,034N,069W,26,NW1/4NW1/4,A,750.00000,,,,,Well,,,External,NWNW,3/2/2019 0:00,2.00000,2.00000,-105.11490,42.89665,CCI,05SC0297516,QTR2,4/10/2020 0:00,210075,,P,W,,,,FREESTONE MIDSTREAM,Miscellaneous - Ground Water
2,wyDG10,Unused Site Record,,11,A,,,P114751.0W,1999-03-31,,Incomplete,"Power Resources, Inc.",,,NE/NW 13-36-73; (59 WELLS),IND_GW,036N,073W,13,NE1/4NW1/4,A5-,885.00000,600.0,335.0,,,Well,,N,External,NENW,3/2/2019 0:00,2.00000,2.00000,-105.56776,43.09796,CCI,05SC0297516,QTR2,4/10/2020 0:00,114751,,P,W,,-1,-1,"Power Resources, Inc.",Industrial - Ground Water
3,wyDG100,,,101,A,,,P205103.0W,2016-02-08,,Incomplete,PEABODY SCHOOL CREEK MINING LLC,,,SCM-EAGLE-1,MIS,041N,070W,6,SE1/4SW1/4,L21,4000.00000,,,,,Well,,,External,SESW,3/2/2019 0:00,2.00000,2.00000,-105.31532,43.55047,CCI,05SC0297516,QTR2,4/10/2020 0:00,205103,,P,W,,,,PEABODY SCHOOL CREEK MINING LLC,Miscellaneous - Ground Water
4,wyDG1000,Unused Site Record,,1001,A,A,,CR UW05/085,1976-05-20,5/20/1976,Fully Adjudicated,,HENRY AND EDNA,SCHNEIDER,SCHNEIDER NO 6 WELL,IRR_GW,029N,061W,29,SW1/4SE1/4,A,700.00000,,,,,Well,Supplemental,,External,SWSE,3/2/2019 0:00,1.00000,1.00000,-104.21279,42.45147,CCI,05SC0297516,QTR2,4/10/2020 0:00,33227,OR 30/242,P,W,,,,"SCHNEIDER, HENRY AND EDNA",Irrigation - Ground Water


In [7]:
# For Wild and Scenic River benuse
# suffix metadata, and anything with a Z in it also has a beneficial use = Wild and Scenic River benuse

def splitWord(word):
    return [char for char in word]

def retrieveWildCenicRiver(word):
    word = str(word)
    worldList = splitWord(word)
    if worldList[0] == "Z":
        outString = "Wild and Scenic River"
    else:
        outString = ""
    
    return outString
    
dfwyg['PreffixWaDEBenUse'] = dfwyg.apply(lambda row: retrieveWildCenicRiver(row['Survey_Type_Survey_Number_Survey_Suffix']), axis=1)
dfwyg['PreffixWaDEBenUse'].unique()

array(['', 'Wild and Scenic River'], dtype=object)

In [8]:
# For Wild and Scenic River benuse
# combine WaDEBenUse & PreffixWaDEBenUse columns if not blank
def combineWaDEBenUses(valA, valB):
    if valA == "" or pd.isnull(valA):
        outString = valB
    else:
        outString = valB + ", " + valA
    
    return outString

dfwyg['WaDEBenUse'] = dfwyg.apply(lambda row: combineWaDEBenUses(row['PreffixWaDEBenUse'], row['WaDEBenUse']), axis=1)
dfwyg['WaDEBenUse'].unique()

array(['Irrigation - Ground Water', 'Miscellaneous - Ground Water',
       'Industrial - Ground Water',
       'Irrigation - Ground Water, Stock Watering',
       'Miscellaneous - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Irrigation - Ground Water, Stock Watering',
       'Municipal - Ground Water',
       'Domestic - Ground Water, Irrigation - Ground Water',
       'Domestic - Ground Water, Industrial - Ground Water, Miscellaneous - Ground Water',
       'Industrial - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Stock Watering',
       'Miscellaneous - Ground Water, Stock Watering',
       'Irrigation - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Irrigation - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Wild and Scenic River',
       'Industrial - Ground Water, Stock Watering',
       'Mi

In [9]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfwyg['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "WYwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "WYwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "WYwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = ""
df['in_WaterSourceNativeID'] = ""
df['in_WaterSourceTypeCV'] = "Groundwater"

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = ""
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfwyg['Latitude_Double']
df['in_Longitude'] = dfwyg['Longitude_Double']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = dfwyg['FacilityName'].str.title().replace(",", " ")
df['in_SiteNativeID'] = "" #auto fill in below
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfwyg['Facility_type']
df['in_StateCV'] = "WY"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfwyg['Total_Flow_CFS___Appropriation_GPM_'].astype(float)
df['in_AllocationLegalStatusCV'] = dfwyg['SummaryWRStatus']
df['in_AllocationNativeID'] =  dfwyg['WR_Number'].replace("", 0).fillna(0).astype(str)
df['in_AllocationOwner'] = dfwyg['WaDEOwner']
df['in_AllocationPriorityDate'] = dfwyg['PriorityDate']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = ""
df['in_BeneficialUseCategory'] = dfwyg['WaDEBenUse']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "http://seoweb.wyo.gov/e-Permit/Common/Login.aspx"

dfground = df.copy()
dfground = dfground.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(dfground))
dfground.head()

2323


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,wyDG0,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,42.91290,-104.46127,,,POD,O W Well #1,,,Well,WY,,,,,,,,,,575.00000,,CR UW04/059,"O. W. RANCH, INC.",1976-08-05,,,,,,Irrigation - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
1,wyDG1,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,42.89665,-105.11490,,,POD,Lincoln Water Well #1,,,Well,WY,,,,,,,,,,750.00000,Incomplete,P210075.0W,FREESTONE MIDSTREAM,2018-11-28,,,,,,Miscellaneous - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
2,wyDG10,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,43.09796,-105.56776,,,POD,Ne/Nw 13-36-73; (59 Wells),,,Well,WY,,,,,,,,,,885.00000,Incomplete,P114751.0W,"Power Resources, Inc.",1999-03-31,,,,,,Industrial - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
3,wyDG100,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,43.55047,-105.31532,,,POD,Scm-Eagle-1,,,Well,WY,,,,,,,,,,4000.00000,Incomplete,P205103.0W,PEABODY SCHOOL CREEK MINING LLC,2016-02-08,,,,,,Miscellaneous - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
4,wyDG1000,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,42.45147,-104.21279,,,POD,Schneider No 6 Well,,,Well,WY,,,,,,,,,,700.00000,Fully Adjudicated,CR UW05/085,"SCHNEIDER, HENRY AND EDNA",1976-05-20,,,,,,Irrigation - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx


### POD Surface Water

In [10]:
# Input File
SW_Input = "POD_SW_DepthI_FC_input.zip"
dfwys = pd.read_csv(SW_Input).replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfwys:
    dfwys['WaDEUUID'] = "wyDS" + dfwys.index.astype(str)
    dfwys.to_csv('POD_SW_DepthI_FC_input.zip', compression=dict(method='zip', archive_name='POD_SW_DepthI_FC_input.csv'), index=False)

print(len(dfwys))
dfwys.head()

9659


,WaDEUUID,ReasonRemoved,IncompleteField,OID_,Contractor,Contract_ID,Horizontal_Accuracy,Creation_Edit_Date,IsActive,SummaryCRStatus,WRPODID,WR_Number,PriorityDate,PriorityText,SummaryWRStatus,Company,FirstName,LastName,FacilityName,Uses,Twn,Rng,Sec,Qtr_Qtr,Survey_Type_Survey_Number_Survey_Suffix,Total_Flow_CFS___Appropriation_GPM_,Stream_Source,Total_Capacity__AF_Yr_,Diversion_Capacity_at_Headgate_CFS_,Facility_type,SupplyType,Chemical_Analysis_Y_N_,CreatedBy,QTRQTR_CLEAN,DateDownloaded,WaterDivision,WaterDistrict,Longitude_Double,Latitude_Double
0,wyDS0,,,1,CCI,05SC0297516,QTR2,4/14/2020 0:00,A,A,,CR CC20/011,12/5/1900,12/5/1900,Fully Adjudicated,BUFFALO CREEK LIVE STOCK COMPANY,,,HOUCK NO. ONE DITCH,IRR_SW,040N,085W,33,SW1/4NW1/4,A,40.05000,Buffalo Creek,,,Stream,Original,,External,SWNW,10/4/2018 0:00,2.00000,8.0,-107.060231,43.393361
1,wyDS1,,,2,CCI,05SC0297516,QTR2,4/14/2020 0:00,A,A,,CR CC23/015,12/5/1900,12/5/1900,Fully Adjudicated,BUFFALO CREEK LIVE STOCK COMPANY,,,HOUCK NO. 2 DITCH,IRR_SW,040N,085W,33,SE1/4SW1/4,A,20.26000,Buffalo Creek,,,Stream,Original,,External,SESW,10/4/2018 0:00,2.00000,8.0,-107.053028,43.386402
2,wyDS10,,,11,CCI,05SC0297516,QTR2,4/14/2020 0:00,A,A,,CR CC33/442,8/12/1905,8/12/1905,Fully Adjudicated,,JAMES,LOCKE,LOCKE DITCH,IRR_SW,041N,106W,6,NW1/4SW1/4,L6,2.04000,Horse Creek,,,Stream,Original,,External,NWSW,10/4/2018 0:00,3.00000,3.0,-109.631786,43.542889
3,wyDS100,,,101,CCI,05SC0297516,QTR2,4/14/2020 0:00,A,,,CD 01/037,07/05/1889,07/05/1889,Fully Adjudicated,,MORETON,FREWON,DICK DITCH,IRR_SW,047N,082W,7,SW1/4SE1/4,A,12.32000,Middle Fork Crazy Woman Creek,,,Stream,,,External,SWSE,10/4/2018 0:00,2.00000,2.0,-106.754161,44.052631
4,wyDS1000,,,1001,CCI,05SC0297516,QTR2,4/14/2020 0:00,A,A,,CR CC20/084,11/12/1898,11/12/1898,Fully Adjudicated,,JOHN,VAN DERVERT,VAN DITCH ACIPT SLOUGH DITCH,IRR_SW,034N,111W,27,NE1/4SW1/4,A,2.28000,Failer Creek,,,Stream,Original,,External,NESW,10/4/2018 0:00,4.00000,11.0,-110.089981,42.887081


In [11]:
#Creating WaDE Owner Field.  
#Create from Owner field. If empty, use LastName + FirstName fields.

def retrieveOwner(Com, FN, LN):
    Com = str(Com).strip()
    FN = str(FN).strip()
    LN = str(LN).strip()
    if Com == "" or pd.isnull(Com):
        outList = LN + ", " + FN
    else:
        outList = Com
    return outList
dfwys['WaDEOwner'] = dfwys.apply(lambda row: retrieveOwner(row['Company'], row['FirstName'], row['LastName']), axis=1)
dfwys['WaDEOwner'].unique()

array(['BUFFALO CREEK LIVE STOCK COMPANY', 'LOCKE, JAMES',
       'FREWON, MORETON', ..., 'JOHNSTON, DAVID', 'MCCONNELL, CHARLES',
       'HILL, S B'], dtype=object)

In [12]:
#Creating Beneficial Use.
#Need to translate WY abbreviatoins to a workable format.

BenUseDict = {
"AESCNG" : "Coal Bed Natural Gas",
"AESFIS" : "Fish Propagation (Aesthetics)",
"AESGWR" : "Ground Water Recharge (Aesthetics)",
"AESREC" : "Recreation (Aesthetics)",
"AESSTK" : "Stock (Aesthetics)",
"AESWET" : "Wetlands (Aesthetics)",
"AESWIL" : "Wildlife (Aesthetics)",
"AQU" : "Aquaculture",
"BOT" : "Bottling Water",
"CAG" : "Commercial Agriculture",
"CBM" : "Coal Bed Methane - Ground Water",
"CHE" : "Chemical",
"CIS" : "Consumptive Instream Flow",
"CMU" : "Combined Uses",
"CNG_SW" : "Coal Bed Natural Gas",
"COM" : "Commercial",
"CUL" : "Culinary",
"DAI" : "Dairy",
"DEW" : "Mine Dewatering",
"DOM_GW" : "Domestic - Ground Water",
"DOM_SW" : "Domestic - Surface Water",
"DPA" : "Domestic (Phase 2 Award)",
"DRI" : "Drilling",
"DSP" : "Domestic Supply",
"DTA" : "Dust Abatement",
"ECAP" : "Existing Capacity",
"ERO" : "Erosion Control",
"FIR" : "Fire Protection",
"FIS" : "Fish Propagation",
"FLO" : "Flood Control",
"FTH" : "Flow Through",
"GWR" : "Ground Water Recharge",
"HEX" : "Heat Extraction",
"HWY" : "Highway Construction",
"HYD" : "Hydropower",
"HYT" : "Hydrostatic Testing",
"ICE" : "Ice Cutting",
"IFA" : "Instream Flow (Phase 2 Award)",
"IND_GW" : "Industrial - Ground Water",
"IND_SW" : "Industrial - Surface Water",
"IRR_GW" : "Irrigation - Ground Water",
"IRR_SW" : "Irrigation - Surface Water",
"ISF" : "Instream Flow",
"LAK" : "Maintain Natural Lake Level (Phase 2 Award)",
"LAW" : "Large Scale Landscape",
"MAI" : "Maintenance (Equipment Washing)",
"MAN" : "Manufacturing",
"MEC" : "Mechanical",
"MED" : "Medicinal",
"MEM" : "Municipal (Emergency)",
"MIL" : "Milling",
"MIN" : "Mining",
"MIS" : "Miscellaneous - Ground Water",
"MON" : "Monitor",
"MUN_GW" : "Municipal - Ground Water",
"MUN_SW" : "Municipal - Surface Water",
"NAT" : "Natural Flow (Phase 2 Award)",
"O&G" : "Oil and Gas Well Drilling",
"OIL" : "Oil",
"OTH" : "Other",
"OTH_CM" : "Other - Commercial",
"OTH_IN" : "Other - Industrial",
"OTH_TM" : "Other - Temporary",
"P&S" : "Potable and Sanitary Supply",
"PCT" : "Pollution Control",
"POW" : "Power",
"RAI" : "Railroad",
"RDC" : "Road Construction",
"REC" : "Recreation",
"REF" : "Refining",
"RES" : "Reservoir Supply",
"REW" : "Reclamation Watering",
"S&D" : "Stock and Domestic",
"SDG" : "Gpm For Domestic or Stock",
"SDU" : "Stock and Domestic",
"SED" : "Sediment Control",
"SNO" : "Snow Making",
"STE" : "Stream",
"STK" : "Stock Watering",
"STKNDMS" : "Stock and Domestic",
"STO" : "Stock",
"STS" : "Stock",
"STW" : "Stock Watering",
"SWD" : "Subdivision",
"SWP" : "Stock Water Pipeline",
"TEM" : "Temporary",
"TENL" : "Total Enlargement",
"TRA" : "Transportation",
"TST" : "Test Well",
"TWR" : "Tree Watering",
"UTL" : "Utilities",
"W&S" : "Wild and Scenic",
"WDR" : "Well Drilling",
"WET" : "Wetlands",
"WHL" : "Water Hauls",
"WL" : "Wildlife"}

def retrieveBenUse(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = ""
    else:
        String1 = str(colrowValue).strip()
        x = String1.split(';')
        try:
            outList = []
            for i in range(len(x)):
                y = x[i].strip()
                y = BenUseDict[y]
                outList.append(y)
            outString = ", ".join(str(e) for e in outList)
        except:
            outString = ""
    return outString

dfwys['WaDEBenUse'] = dfwys.apply(lambda row: retrieveBenUse(row['Uses']), axis=1)
dfwys['WaDEBenUse'].unique()

array(['Irrigation - Surface Water', 'Irrigation - Surface Water, Stock',
       'Irrigation - Surface Water, Stock and Domestic', '',
       'Domestic - Surface Water, Irrigation - Surface Water',
       'Domestic - Surface Water, Irrigation - Surface Water, Stock',
       'Other', 'Flow Through, Reservoir Supply', 'Reservoir Supply',
       'Stock', 'Power', 'Irrigation - Surface Water, Reservoir Supply',
       'Instream Flow', 'Fish Propagation, Reservoir Supply',
       'Domestic - Surface Water, Irrigation - Surface Water, Reservoir Supply',
       'Hydropower', 'Recreation', 'Fish Propagation',
       'Irrigation - Surface Water, Other', 'Wetlands',
       'Domestic - Surface Water, Irrigation - Surface Water, Stock and Domestic',
       'Irrigation - Surface Water, Domestic - Surface Water',
       'Industrial - Surface Water',
       'Domestic Supply, Irrigation - Surface Water',
       'Domestic - Surface Water', 'Municipal - Surface Water',
       'Irrigation - Surface Water

In [13]:
# For Wild and Scenic River benuse
# suffix metadata, and anything with a Z in it also has a beneficial use = Wild and Scenic River benuse

def splitWord(word):
    return [char for char in word]

def retrieveWildCenicRiver(word):
    word = str(word)
    worldList = splitWord(word)
    try:
        if worldList[0] == "Z":
            outString = "Wild and Scenic River"
        else:
            outString = ""
    except:
        outString = ""
    
    return outString
    
dfwys['PreffixWaDEBenUse'] = dfwys.apply(lambda row: retrieveWildCenicRiver(row['Survey_Type_Survey_Number_Survey_Suffix']), axis=1)
dfwys['PreffixWaDEBenUse'].unique()

array(['', 'Wild and Scenic River'], dtype=object)

In [14]:
# For Wild and Scenic River benuse
# combine WaDEBenUse & PreffixWaDEBenUse columns if not blank
def combineWaDEBenUses(valA, valB):
    if valA == "" or pd.isnull(valA):
        outString = valB
    else:
        outString = valB + ", " + valA
    
    return outString

dfwys['WaDEBenUse'] = dfwys.apply(lambda row: combineWaDEBenUses(row['PreffixWaDEBenUse'], row['WaDEBenUse']), axis=1)
dfwys['WaDEBenUse'].unique()

array(['Irrigation - Surface Water', 'Irrigation - Surface Water, Stock',
       'Irrigation - Surface Water, Stock and Domestic', '',
       'Domestic - Surface Water, Irrigation - Surface Water',
       'Domestic - Surface Water, Irrigation - Surface Water, Stock',
       'Other', 'Flow Through, Reservoir Supply', 'Reservoir Supply',
       'Stock', 'Power', 'Irrigation - Surface Water, Reservoir Supply',
       'Instream Flow', 'Fish Propagation, Reservoir Supply',
       'Domestic - Surface Water, Irrigation - Surface Water, Reservoir Supply',
       'Hydropower', 'Recreation', 'Fish Propagation',
       'Irrigation - Surface Water, Stock, Wild and Scenic River',
       'Irrigation - Surface Water, Other', 'Wetlands',
       'Domestic - Surface Water, Irrigation - Surface Water, Stock and Domestic',
       'Irrigation - Surface Water, Domestic - Surface Water',
       'Industrial - Surface Water',
       'Domestic Supply, Irrigation - Surface Water',
       'Domestic - Surface Wate

In [15]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfwys['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "WYwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "WYwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "WYwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfwys['Stream_Source'].str.title().replace(",", " ")
df['in_WaterSourceNativeID'] = ""
df['in_WaterSourceTypeCV'] = "Surface Water"

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = ""
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfwys['Latitude_Double']
df['in_Longitude'] = dfwys['Longitude_Double']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = dfwys['FacilityName'].str.title().replace(",", " ")
df['in_SiteNativeID'] = "" #auto fill in below
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfwys['Facility_type']
df['in_StateCV'] = "WY"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfwys['Total_Flow_CFS___Appropriation_GPM_'].astype(float)
df['in_AllocationLegalStatusCV'] = dfwys['SummaryWRStatus']
df['in_AllocationNativeID'] =  dfwys['WR_Number'].replace("", 0).fillna(0).astype(str)
df['in_AllocationOwner'] = dfwys['WaDEOwner']
df['in_AllocationPriorityDate'] = dfwys['PriorityDate']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = ""
df['in_BeneficialUseCategory'] = dfwys['WaDEBenUse']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "http://seoweb.wyo.gov/e-Permit/Common/Login.aspx"

df_Surface = df.copy()
df_Surface = df_Surface.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(df_Surface))
df_Surface.head()

9659


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,wyDS0,WYwr_M1,WYwr_V1,WYwr_O1,,,,Buffalo Creek,,Surface Water,,,,4326,,,,43.393361,-107.060231,,,POD,Houck No. One Ditch,,,Stream,WY,,,,,,,,,,40.05000,Fully Adjudicated,CR CC20/011,BUFFALO CREEK LIVE STOCK COMPANY,12/5/1900,,,,,,Irrigation - Surface Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
1,wyDS1,WYwr_M1,WYwr_V1,WYwr_O1,,,,Buffalo Creek,,Surface Water,,,,4326,,,,43.386402,-107.053028,,,POD,Houck No. 2 Ditch,,,Stream,WY,,,,,,,,,,20.26000,Fully Adjudicated,CR CC23/015,BUFFALO CREEK LIVE STOCK COMPANY,12/5/1900,,,,,,Irrigation - Surface Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
2,wyDS10,WYwr_M1,WYwr_V1,WYwr_O1,,,,Horse Creek,,Surface Water,,,,4326,,,,43.542889,-109.631786,,,POD,Locke Ditch,,,Stream,WY,,,,,,,,,,2.04000,Fully Adjudicated,CR CC33/442,"LOCKE, JAMES",8/12/1905,,,,,,Irrigation - Surface Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
3,wyDS100,WYwr_M1,WYwr_V1,WYwr_O1,,,,Middle Fork Crazy Woman Creek,,Surface Water,,,,4326,,,,44.052631,-106.754161,,,POD,Dick Ditch,,,Stream,WY,,,,,,,,,,12.32000,Fully Adjudicated,CD 01/037,"FREWON, MORETON",07/05/1889,,,,,,Irrigation - Surface Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
4,wyDS1000,WYwr_M1,WYwr_V1,WYwr_O1,,,,Failer Creek,,Surface Water,,,,4326,,,,42.887081,-110.089981,,,POD,Van Ditch Acipt Slough Ditch,,,Stream,WY,,,,,,,,,,2.28000,Fully Adjudicated,CR CC20/084,"VAN DERVERT, JOHN",11/12/1898,,,,,,Irrigation - Surface Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx


# POU Data

In [16]:
# Input File
POU_Input = "POU_input.zip"
dfinPOU = pd.read_csv(POU_Input).replace(np.nan, "").replace("nan,nan", "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOU:
    dfinPOU['WaDEUUID'] = "wyU" + dfinPOU.index.astype(str)
    dfinPOU.to_csv('POU_input.zip', compression=dict(method='zip', archive_name='POU_input.csv'), index=False)

dfinPOU = dfinPOU.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(dfinPOU))
dfinPOU.head()

C:\Users\rjame\AppData\Local\Temp\ipykernel_1808\651086354.py:3: DtypeWarning: Columns (11,20,29,39,42,45,46,47,48,51,52,53,54,55,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  dfinPOU = pd.read_csv(POU_Input).replace(np.nan, "").replace("nan,nan", "")


491676


,WaDEUUID,ReasonRemoved,IncompleteField,OID_,ContractID,Contractor,Creation_Edit_Date,Horizontal_Accuracy,WRID,Duplicate_ID,Source_ID,PermitNumber,OrderRecordNumber,PermitPrefix,PermitSuffix,IsActive,SummaryCRStatus,WRNumber,PriorityDate,PriorityText,Priority_YR,SummaryWRStatus,Company,FirstName,LastName,FacilityName,Uses,Twn,Rng,Sec,POUQTR,QtrQtr,Survey,POUStatus,SupplyType,CreatedBy,Latitude,Longitude,DateDownloaded,WaterDivision,WaterDistrict,QTRQTR_CLEAN,PLSS_SECTION,PLSS_QTR,PLSS_QTRQTR,Acres,Mapping_Level,Agg_POU_Count,PLSS_TractLot,SupplySource,FacilityType,Centroid_X,Centroid_Y,AcresGISpoly,AcresDifference,ErrorType,Notes,Shape_Length,Shape_Area
0,wyU0,,,1,,MIKE.ROBERTSON,3/6/2021 0:13,,,-3.0-OR 17/250IRR_SWOriginal033N099W05SWSW1/4S...,-3.0-,3.00000,OR 17/250,-,-,,,OR 17/250,1877-06-20,Spring1877,1877.00000,,,HOWARD E.,MCCLINTOCK,St. John Ditch,IRR_SW,033N,099W,5.00000,SW,SW1/4SW1/4,A11-,ADJUDICATED,Original,,42.84638,-108.73479,10/31/2018 6:00,3.00000,1,SWSW,,,033N099W05SWSW,17.42,0.00000,1.00000,,,,-108.7229992,42.85544126,40.10182,-22.68181952,,,0.01711,0.00002
1,wyU1,,,2,,MIKE.ROBERTSON,3/6/2021 0:13,,,-3.0-OR 17/250IRR_SWOriginal033N099W08NWNW1/4N...,-3.0-,3.00000,OR 17/250,-,-,,,OR 17/250,1877-06-20,Spring1877,1877.00000,,,HOWARD E.,MCCLINTOCK,St. John Ditch,IRR_SW,033N,099W,8.00000,NW,NW1/4NW1/4,,ADJUDICATED,Original,,42.84638,-108.73479,6/11/2018 6:00,3.00000,1,NWNW,,,033N099W08NWNW,17.9,0.00000,1.00000,,,,-108.7228458,42.85181,40.22162,-22.3216157,,,0.01713,0.00002
2,wyU10,,,11,,MIKE.ROBERTSON,3/6/2021 0:13,,,A1002.0QA1002.0QIRR_SWOriginal007N005W06SESW1/...,A1002.0Q,1002.00000,,A,Q,,,A1002.0Q,1868-07-03,07/03/1868,1868.00000,Fully Adjudicated,EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES,,,MCKISSOCK DITCH,IRR_SW,007N,005W,6.00000,SE,SW1/4SE1/4,A,ADJUDICATED,Original,,43.60791111,-109.3985306,7/11/2018 6:00,3.00000,3,SWSE,,,007N005W06SWSE,4.3,0.00000,1.00000,,,,-109.3974284,43.6074618,30.84839,-26.54839421,,,0.01748,0.00001
3,wyU100,Incomplete or bad entry for IrrigationMethodCV...,"0.7,1.6,2.6,1.0,4.8,5.0,6.0,7.0,6.2,3.0,10.1,8...",101,,MIKE.ROBERTSON,3/6/2021 0:13,,,A101.0QA101.0QIRR_SWOriginal001S001E29NENE1/4N...,A101.0Q,101.00000,,A,Q,,,A101.0Q,1868-07-03,07/03/1868,1868.00000,Fully Adjudicated,EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES,,,RAY DITCH,IRR_SW,001S,001E,29.00000,NE,NE1/4NE1/4,A,ADJUDICATED,Original,,43.00093889,-108.9303694,11/14/2018 7:00,3.00000,1,NENE,,,001S001E29NENE,26.0,0.00000,1.00000,,,,-108.7774292,42.95154059,39.96107,-13.96107119,,,0.01710,0.00002
4,wyU1000,Incomplete or bad entry for IrrigationMethodCV...,"0.6,1.2,1.0,2.0,5.0,8.0,10.0,11.0,12.0,12.4,13...",1001,,MIKE.ROBERTSON,3/6/2021 0:13,,,A1910.0QA1910.0QIRR_SWOriginal003N002W06SWNE1/...,A1910.0Q,1910.00000,,A,Q,,,A1910.0Q,1868-07-03,7/3/1868,1868.00000,Fully Adjudicated,EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES,,,CHARLES STAGNER DITCH,IRR_SW,003N,002W,6.00000,SW,NE1/4SW1/4,A,ADJUDICATED,Original,,43.27763339,-109.082517,10/31/2018 6:00,3.00000,3,NESW,,,003N002W06NESW,16.0,0.00000,1.00000,,,,-109.0440231,43.26281467,40.15107,-24.15106873,,,0.01718,0.00002


In [17]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
dfinPOU['PriorityDate'] = pd.to_datetime(dfinPOU['PriorityDate'])
dfinPOU['PriorityDate'] = pd.to_datetime(dfinPOU["PriorityDate"].dt.strftime('%m/%d/%Y'))
dfinPOU['PriorityDate'].unique()

array(['1877-06-20T00:00:00.000000000', '1868-07-03T00:00:00.000000000',
       '1897-08-23T00:00:00.000000000', ...,
       '1896-10-22T00:00:00.000000000', '1897-06-15T00:00:00.000000000',
       '1897-04-13T00:00:00.000000000'], dtype='datetime64[ns]')

In [18]:
#Creating WaDE Owner Field.  
#Create from Owner field. If empty, use LastName + FirstName fields.

def retrieveOwner(Com, FN, LN):
    Com = str(Com).strip()
    FN = str(FN).strip()
    LN = str(LN).strip()
    if Com == "" or pd.isnull(Com):
        outList = LN + ", " + FN
    else:
        outList = Com
    return outList
dfinPOU['WaDEOwner'] = dfinPOU.apply(lambda row: retrieveOwner(row['Company'], row['FirstName'], row['LastName']), axis=1)
dfinPOU['WaDEOwner'].unique()

array(['MCCLINTOCK, HOWARD E.',
       'EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES',
       'PAHLOW, ROBERT AND WILHELMINE', ..., 'COLE, S. J.',
       'RITTERLING, HENRY', 'RUBY, FRANK'], dtype=object)

In [19]:
#Creating Beneficial Use.
#Need to translate WY abbreviatoins to a workable format.

BenUseDict = {
"AESCNG" : "Coal Bed Natural Gas",
"AESFIS" : "Fish Propagation (Aesthetics)",
"AESGWR" : "Ground Water Recharge (Aesthetics)",
"AESREC" : "Recreation (Aesthetics)",
"AESSTK" : "Stock (Aesthetics)",
"AESWET" : "Wetlands (Aesthetics)",
"AESWIL" : "Wildlife (Aesthetics)",
"AQU" : "Aquaculture",
"BOT" : "Bottling Water",
"CAG" : "Commercial Agriculture",
"CBM" : "Coal Bed Methane - Ground Water",
"CHE" : "Chemical",
"CIS" : "Consumptive Instream Flow",
"CMU" : "Combined Uses",
"CNG_SW" : "Coal Bed Natural Gas",
"COM" : "Commercial",
"CUL" : "Culinary",
"DAI" : "Dairy",
"DEW" : "Mine Dewatering",
"DOM_GW" : "Domestic - Ground Water",
"DOM_SW" : "Domestic - Surface Water",
"DPA" : "Domestic (Phase 2 Award)",
"DRI" : "Drilling",
"DSP" : "Domestic Supply",
"DTA" : "Dust Abatement",
"ECAP" : "Existing Capacity",
"ERO" : "Erosion Control",
"FIR" : "Fire Protection",
"FIS" : "Fish Propagation",
"FLO" : "Flood Control",
"FTH" : "Flow Through",
"GWR" : "Ground Water Recharge",
"HEX" : "Heat Extraction",
"HWY" : "Highway Construction",
"HYD" : "Hydropower",
"HYT" : "Hydrostatic Testing",
"ICE" : "Ice Cutting",
"IFA" : "Instream Flow (Phase 2 Award)",
"IND_GW" : "Industrial - Ground Water",
"IND_SW" : "Industrial - Surface Water",
"IRR_GW" : "Irrigation - Ground Water",
"IRR_SW" : "Irrigation - Surface Water",
"ISF" : "Instream Flow",
"LAK" : "Maintain Natural Lake Level (Phase 2 Award)",
"LAW" : "Large Scale Landscape",
"MAI" : "Maintenance (Equipment Washing)",
"MAN" : "Manufacturing",
"MEC" : "Mechanical",
"MED" : "Medicinal",
"MEM" : "Municipal (Emergency)",
"MIL" : "Milling",
"MIN" : "Mining",
"MIS" : "Miscellaneous - Ground Water",
"MON" : "Monitor",
"MUN_GW" : "Municipal - Ground Water",
"MUN_SW" : "Municipal - Surface Water",
"NAT" : "Natural Flow (Phase 2 Award)",
"O&G" : "Oil and Gas Well Drilling",
"OIL" : "Oil",
"OTH" : "Other",
"OTH_CM" : "Other - Commercial",
"OTH_IN" : "Other - Industrial",
"OTH_TM" : "Other - Temporary",
"P&S" : "Potable and Sanitary Supply",
"PCT" : "Pollution Control",
"POW" : "Power",
"RAI" : "Railroad",
"RDC" : "Road Construction",
"REC" : "Recreation",
"REF" : "Refining",
"RES" : "Reservoir Supply",
"REW" : "Reclamation Watering",
"S&D" : "Stock and Domestic",
"SDG" : "Gpm For Domestic or Stock",
"SDU" : "Stock and Domestic",
"SED" : "Sediment Control",
"SNO" : "Snow Making",
"STE" : "Stream",
"STK" : "Stock Watering",
"STKNDMS" : "Stock and Domestic",
"STO" : "Stock",
"STS" : "Stock",
"STW" : "Stock Watering",
"SWD" : "Subdivision",
"SWP" : "Stock Water Pipeline",
"TEM" : "Temporary",
"TENL" : "Total Enlargement",
"TRA" : "Transportation",
"TST" : "Test Well",
"TWR" : "Tree Watering",
"UTL" : "Utilities",
"W&S" : "Wild and Scenic",
"WDR" : "Well Drilling",
"WET" : "Wetlands",
"WHL" : "Water Hauls",
"WL" : "Wildlife"}

def retrieveBenUse(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = ""
    else:
        String1 = str(colrowValue).strip()
        x = String1.split(';')
        try:
            outList = []
            for i in range(len(x)):
                y = x[i].strip()
                y = BenUseDict[y]
                outList.append(y)
            outString = ", ".join(str(e) for e in outList)
        except:
            outString = ""
    return outString

dfinPOU['WaDEBenUse'] = dfinPOU.apply(lambda row: retrieveBenUse(row['Uses']), axis=1)
dfinPOU['WaDEBenUse'].unique()

array(['Irrigation - Surface Water', 'Municipal - Surface Water',
       'Coal Bed Methane - Ground Water',
       'Coal Bed Methane - Ground Water, Miscellaneous - Ground Water',
       'Industrial - Surface Water, Irrigation - Surface Water',
       'Wildlife', 'Domestic - Surface Water, Irrigation - Surface Water',
       'Fish Propagation',
       'Coal Bed Methane - Ground Water, Stock Watering',
       'Industrial - Surface Water',
       'Combined Uses, Stock, Wetlands, Wildlife',
       'Irrigation - Ground Water',
       'Domestic - Ground Water, Industrial - Ground Water',
       'Miscellaneous - Ground Water', 'Flood Control', 'Other',
       'Stock, Wildlife', 'Industrial - Surface Water, Stock, Wildlife',
       'Fire Protection, Fish Propagation',
       'Irrigation - Surface Water, Stock',
       'Irrigation - Surface Water, Stock and Domestic',
       'Irrigation - Ground Water, Miscellaneous - Ground Water, Stock Watering',
       'Domestic - Ground Water, Miscellaneou

In [20]:
# For Wild and Scenic River benuse
# suffix metadata, and anything with a Z in it also has a beneficial use = Wild and Scenic River benuse

dfinPOU['Survey'] = dfinPOU['Survey'].replace("", 0).fillna(0).astype(str)

def splitWord(word):
    return [char for char in word]

def retrieveWildCenicRiver(word):
    word = str(word)
    worldList = splitWord(word)
    if worldList[0] == "Z":
        outString = "Wild and Scenic River"
    else:
        outString = ""
    
    return outString
    
dfinPOU['PreffixWaDEBenUse'] = dfinPOU.apply(lambda row: retrieveWildCenicRiver(row['Survey']), axis=1)
dfinPOU['PreffixWaDEBenUse'] .unique()

array(['', 'Wild and Scenic River'], dtype=object)

In [21]:
# For Wild and Scenic River benuse
# combine WaDEBenUse & PreffixWaDEBenUse columns if not blank
def combineWaDEBenUses(valA, valB):
    if valA == "" or pd.isnull(valA):
        outString = valB
    else:
        outString = valB + ", " + valA
    
    return outString

dfinPOU['WaDEBenUse'] = dfinPOU.apply(lambda row: combineWaDEBenUses(row['PreffixWaDEBenUse'], row['WaDEBenUse']), axis=1)
dfinPOU['WaDEBenUse'].unique()

array(['Irrigation - Surface Water', 'Municipal - Surface Water',
       'Coal Bed Methane - Ground Water',
       'Coal Bed Methane - Ground Water, Miscellaneous - Ground Water',
       'Industrial - Surface Water, Irrigation - Surface Water',
       'Wildlife', 'Domestic - Surface Water, Irrigation - Surface Water',
       'Fish Propagation',
       'Coal Bed Methane - Ground Water, Stock Watering',
       'Industrial - Surface Water',
       'Combined Uses, Stock, Wetlands, Wildlife',
       'Irrigation - Ground Water',
       'Domestic - Ground Water, Industrial - Ground Water',
       'Miscellaneous - Ground Water', 'Flood Control', 'Other',
       'Stock, Wildlife', 'Industrial - Surface Water, Stock, Wildlife',
       'Fire Protection, Fish Propagation',
       'Irrigation - Surface Water, Stock',
       'Irrigation - Surface Water, Stock and Domestic',
       'Irrigation - Ground Water, Miscellaneous - Ground Water, Stock Watering',
       'Domestic - Ground Water, Miscellaneou

In [22]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOU['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "WYwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "WYwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "WYwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOU['SupplySource'].str.title().replace(",", " ")
df['in_WaterSourceNativeID'] = ""
df['in_WaterSourceTypeCV'] = ""

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = ""
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOU['Latitude']
df['in_Longitude'] = dfinPOU['Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POU"
df['in_SiteName'] = dfinPOU['FacilityName'].str.title().replace(",", " ")
df['in_SiteNativeID'] = "" #auto fill in below
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfinPOU['FacilityType']
df['in_StateCV'] = "WY"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = ""
df['in_AllocationLegalStatusCV'] = dfinPOU['SummaryWRStatus']
df['in_AllocationNativeID'] =  dfinPOU['WRNumber'].replace("", 0).fillna(0).astype(str)
df['in_AllocationOwner'] = dfinPOU['WaDEOwner']
df['in_AllocationPriorityDate'] = dfinPOU['PriorityDate']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = ""
df['in_BeneficialUseCategory'] = dfinPOU['WaDEBenUse']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = dfinPOU['Acres']
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "http://seoweb.wyo.gov/e-Permit/Common/Login.aspx"

outPOU = df.copy()
outPOU = outPOU.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outPOU))
outPOU.head()

491676


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,wyU0,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,,,,,4326,,,,42.84638,-108.73479,,,POU,St. John Ditch,,,,WY,,,,,,,,,,,,OR 17/250,"MCCLINTOCK, HOWARD E.",1877-06-20,,,,,,Irrigation - Surface Water,,,,,,0,,,17.42,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
1,wyU1,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,,,,,4326,,,,42.84638,-108.73479,,,POU,St. John Ditch,,,,WY,,,,,,,,,,,,OR 17/250,"MCCLINTOCK, HOWARD E.",1877-06-20,,,,,,Irrigation - Surface Water,,,,,,0,,,17.9,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
2,wyU10,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,,,,,4326,,,,43.60791111,-109.3985306,,,POU,Mckissock Ditch,,,,WY,,,,,,,,,,,Fully Adjudicated,A1002.0Q,EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES,1868-07-03,,,,,,Irrigation - Surface Water,,,,,,0,,,4.3,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
3,wyU100,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,,,,,4326,,,,43.00093889,-108.9303694,,,POU,Ray Ditch,,,,WY,,,,,,,,,,,Fully Adjudicated,A101.0Q,EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES,1868-07-03,,,,,,Irrigation - Surface Water,,,,,,0,,,26.0,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
4,wyU1000,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,,,,,4326,,,,43.27763339,-109.082517,,,POU,Charles Stagner Ditch,,,,WY,,,,,,,,,,,Fully Adjudicated,A1910.0Q,EASTERN SHOSHONE AND NORTHERN ARAPAHO TRIBES,1868-07-03,,,,,,Irrigation - Surface Water,,,,,,0,,,16.0,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx


# Concatenate POD & POU

In [23]:
# Concatenate
frames = [dfground, df_Surface, outPOU]
outdf = pd.concat(frames)
outdf = outdf.replace(np.nan, "").drop_duplicates()
outdf = outdf.replace("nan", "").drop_duplicates()

print(len(outdf))
outdf

503658


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,wyDG0,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,42.91290,-104.46127,,,POD,O W Well #1,,,Well,WY,,,,,,,,,,575.00000,,CR UW04/059,"O. W. RANCH, INC.",1976-08-05 00:00:00,,,,,,Irrigation - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
1,wyDG1,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,42.89665,-105.11490,,,POD,Lincoln Water Well #1,,,Well,WY,,,,,,,,,,750.00000,Incomplete,P210075.0W,FREESTONE MIDSTREAM,2018-11-28 00:00:00,,,,,,Miscellaneous - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
2,wyDG10,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,43.09796,-105.56776,,,POD,Ne/Nw 13-36-73; (59 Wells),,,Well,WY,,,,,,,,,,885.00000,Incomplete,P114751.0W,"Power Resources, Inc.",1999-03-31 00:00:00,,,,,,Industrial - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
3,wyDG100,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,43.55047,-105.31532,,,POD,Scm-Eagle-1,,,Well,WY,,,,,,,,,,4000.00000,Incomplete,P205103.0W,PEABODY SCHOOL CREEK MINING LLC,2016-02-08 00:00:00,,,,,,Miscellaneous - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
4,wyDG1000,WYwr_M1,WYwr_V1,WYwr_O1,,,,,,Groundwater,,,,4326,,,,42.45147,-104.21279,,,POD,Schneider No 6 Well,,,Well,WY,,,,,,,,,,700.00000,Fully Adjudicated,CR UW05/085,"SCHNEIDER, HENRY AND EDNA",1976-05-20 00:00:00,,,,,,Irrigation - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491671,wyU99995,WYwr_M1,WYwr_V1,WYwr_O1,,,,North Platte River,,,,,,4326,,,,42.83176111,-106.3690611,,,POU,Midwest Refining Company Pipeline No 2 Act Amo...,,,,WY,,,,,,,,,,,Fully Adjudicated,CR CC97/058,CITY OF CASPER,1917-10-30 00:00:00,,,,,,Municipal - Surface Water,,,,,,0,,,0.00000,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
491672,wyU99996,WYwr_M1,WYwr_V1,WYwr_O1,,,,North Platte River,,,,,,4326,,,,42.83176111,-106.3690611,,,POU,Midwest Refining Company Pipeline No 2 Act Amo...,,,,WY,,,,,,,,,,,Fully Adjudicated,CR CC97/058,CITY OF CASPER,1917-10-30 00:00:00,,,,,,Municipal - Surface Water,,,,,,0,,,0.00000,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
491673,wyU99997,WYwr_M1,WYwr_V1,WYwr_O1,,,,North Platte River,,,,,,4326,,,,42.83176111,-106.3690611,,,POU,Midwest Refining Company Pipeline No 2 Act Amo...,,,,WY,,,,,,,,,,,Fully Adjudicated,CR CC97/058,CITY OF C

## Clean Data / data types

In [24]:
# Clean name entries of spcial characters
def removeSpecialCharsFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;/\)(-]", "", Val).title().replace("  ", " ").strip()
    return Val

In [25]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array(['', 'Buffalo Creek', 'Horse Creek', ...,
       'North Platte River, Pathfinder And Guernsey Reservoirs',
       'North Platte River And Pathfinder Reservoir Permit No 609 Res',
       '<Null>'], dtype=object)

In [26]:
outdf['in_SiteName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['O W Well #1', 'Lincoln Water Well #1', 'Nenw 133673 59 Wells',
       ..., 'Arfederal 2089Nw28',
       'Enl Scissors Ditch As Changed To Grayrocks Reservoir Pump Station',
       'Ritterling Ditch As Changed To The Ritterling No 1 Pump And Pipeline'],
      dtype=object)

In [27]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['O W Ranch, Inc', 'Freestone Midstream', 'Power Resources, Inc',
       ..., 'Impact Exploration Production, Llc', 'Malody, Sharon',
       'Ritterling, Henry'], dtype=object)

In [28]:
def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [29]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array(['', 'Buffalo Creek', 'Horse Creek', ...,
       'North Platte River, Pathfinder And Guernsey Reservoirs',
       'North Platte River And Pathfinder Reservoir Permit No 609 Res',
       '<Null>'], dtype=object)

In [30]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['Groundwater', 'Surface Water', ''], dtype=object)

In [31]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['O W Well #1', 'Lincoln Water Well #1', 'Nenw 133673 59 Wells',
       ..., 'Arfederal 2089Nw28',
       'Enl Scissors Ditch As Changed To Grayrocks Reservoir Pump Station',
       'Ritterling Ditch As Changed To The Ritterling No 1 Pump And Pipeline'],
      dtype=object)

In [32]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['Well', 'Stream', 'Spring', ''], dtype=object)

In [33]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['', 'Incomplete', 'Fully Adjudicated', 'Complete', 'Unadjudicated',
       'Partially Adjudicated', 'Suspended', 'Rejected'], dtype=object)

In [34]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['O W Ranch, Inc', 'Freestone Midstream', 'Power Resources, Inc',
       ..., 'Impact Exploration Production, Llc', 'Malody, Sharon',
       'Ritterling, Henry'], dtype=object)

In [35]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array(['Irrigation - Ground Water', 'Miscellaneous - Ground Water',
       'Industrial - Ground Water',
       'Irrigation - Ground Water, Stock Watering',
       'Miscellaneous - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Irrigation - Ground Water, Stock Watering',
       'Municipal - Ground Water',
       'Domestic - Ground Water, Irrigation - Ground Water',
       'Domestic - Ground Water, Industrial - Ground Water, Miscellaneous - Ground Water',
       'Industrial - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Stock Watering',
       'Miscellaneous - Ground Water, Stock Watering',
       'Irrigation - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Irrigation - Ground Water, Miscellaneous - Ground Water',
       'Domestic - Ground Water, Wild and Scenic River',
       'Industrial - Ground Water, Stock Watering',
       'Mi

In [36]:
# in_Latitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').replace(0,"").fillna("")
outdf['in_Latitude'].unique()

array([42.9129, 42.89665, 43.09796, ..., 41.14201944, 41.22348889,
       42.12836389], dtype=object)

In [37]:
# in_Longitude
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').replace(0,"").fillna("")
outdf['in_Longitude'].unique()

array([-104.46127, -105.1149, -105.56776, ..., -105.9667694, -105.7206611,
       -104.8890028], dtype=object)

In [38]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'], errors = 'coerce')
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

array(['1976-08-05T00:00:00.000000000', '2018-11-28T00:00:00.000000000',
       '1999-03-31T00:00:00.000000000', ...,
       '2002-12-20T00:00:00.000000000', '1897-06-15T00:00:00.000000000',
       '1897-04-13T00:00:00.000000000'], dtype='datetime64[ns]')

In [39]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

array([575.0, 750.0, 885.0, ..., 7.28, 99.66, 12.03], dtype=object)

In [40]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

array([''], dtype=object)

In [41]:
# Fixing in_IrrigatedAcreage datatype
outdf['in_IrrigatedAcreage'] = pd.to_numeric(outdf['in_IrrigatedAcreage'], errors='coerce').replace(0,"").fillna("")
outdf['in_IrrigatedAcreage'].unique()

array([''], dtype=object)

## WaDE Custom Elements (due to missing state site info)

In [42]:
# Creating WaDE Custom site native ID for easy site identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp SiteNativeID dataframe of unique site.
def assignSiteUUID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfSiteNativeID = pd.DataFrame()
dfSiteNativeID['in_Latitude'] = outdf['in_Latitude']
dfSiteNativeID['in_Longitude'] = outdf['in_Longitude']
dfSiteNativeID['in_SiteName'] = outdf['in_SiteName'].astype(str).str.strip()
dfSiteNativeID['in_PODorPOUSite'] = outdf['in_PODorPOUSite'].astype(str).str.strip()
dfSiteNativeID = dfSiteNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfSiteNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfSiteNativeID['in_SiteNativeID'] = dftemp.apply(lambda row: assignSiteUUID(row['Count']), axis=1)
dfSiteNativeID['linkKey'] = dfSiteNativeID['in_Latitude'].astype(str) + dfSiteNativeID['in_Longitude'].astype(str) + dfSiteNativeID['in_SiteName'].astype(str) + dfSiteNativeID['in_PODorPOUSite'].astype(str)
# # ----------------------------------------------------------------------------------------------------

SiteNativeIDdict = pd.Series(dfSiteNativeID.in_SiteNativeID.values, index=dfSiteNativeID.linkKey.astype(str)).to_dict()
def retrieveSiteNativeID(A, B, C, D):
    colrowValue = str(A).strip() + str(B).strip() + str(C).strip() + str(D).strip()
    outList = SiteNativeIDdict[colrowValue]
    return outList


outdf['in_SiteNativeID'] = outdf.apply(lambda row: retrieveSiteNativeID( row['in_Latitude'], row['in_Longitude'], row['in_SiteName'], row['in_PODorPOUSite']), axis=1)
outdf['in_SiteNativeID'].unique()

array(['wadeID1', 'wadeID2', 'wadeID3', ..., 'wadeID76003', 'wadeID76004',
       'wadeID76005'], dtype=object)

In [43]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName'].astype(str).str.strip()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV'].astype(str).str.strip()
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    colrowValue = str(A).strip() + str(B).strip()
    outList = WaterSourceNativeIDdict[colrowValue]
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

array(['wadeID1', 'wadeID2', 'wadeID3', ..., 'wadeID2592', 'wadeID2593',
       'wadeID2594'], dtype=object)

## Drop non-Active AllocationLegalStatusCV Water Rights
- For this {state name / organization}, we don't want water rights that are considered: {enter string entries here}

Note:
- none identified for WY, at this time.

In [44]:
# # drop non-active AllocationLegalStatusCV values specific to that state.

# # drop the list
# dropLegalStatusList = [""] # enter string entries here

# # drop rows from above list
# outdf = outdf[outdf.in_AllocationLegalStatusCV.isin(dropLegalStatusList) == False].reset_index(drop=True)

# print(len(outdf))
# outdf['in_AllocationLegalStatusCV'].unique()

## Export Outputs

In [45]:
outdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503658 entries, 0 to 491675
Data columns (total 63 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   WaDEUUID                                      503658 non-null  object        
 1   in_MethodUUID                                 503658 non-null  object        
 2   in_VariableSpecificUUID                       503658 non-null  object        
 3   in_OrganizationUUID                           503658 non-null  object        
 4   in_Geometry                                   503658 non-null  object        
 5   in_GNISFeatureNameCV                          503658 non-null  object        
 6   in_WaterQualityIndicatorCV                    503658 non-null  object        
 7   in_WaterSourceName                            503658 non-null  object        
 8   in_WaterSourceNativeID                        503658 n

In [46]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,wyDG0,WYwr_M1,WYwr_V1,WYwr_O1,,,,,wadeID1,Groundwater,,,,4326,,,,42.91290,-104.46127,,,POD,O W Well #1,wadeID1,,Well,WY,,,,,,,,,,575.00000,,CR UW04/059,"O W Ranch, Inc",1976-08-05,,,,,,Irrigation - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
1,wyDG1,WYwr_M1,WYwr_V1,WYwr_O1,,,,,wadeID1,Groundwater,,,,4326,,,,42.89665,-105.11490,,,POD,Lincoln Water Well #1,wadeID2,,Well,WY,,,,,,,,,,750.00000,Incomplete,P210075.0W,Freestone Midstream,2018-11-28,,,,,,Miscellaneous - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
2,wyDG10,WYwr_M1,WYwr_V1,WYwr_O1,,,,,wadeID1,Groundwater,,,,4326,,,,43.09796,-105.56776,,,POD,Nenw 133673 59 Wells,wadeID3,,Well,WY,,,,,,,,,,885.00000,Incomplete,P114751.0W,"Power Resources, Inc",1999-03-31,,,,,,Industrial - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
3,wyDG100,WYwr_M1,WYwr_V1,WYwr_O1,,,,,wadeID1,Groundwater,,,,4326,,,,43.55047,-105.31532,,,POD,Scmeagle1,wadeID4,,Well,WY,,,,,,,,,,4000.00000,Incomplete,P205103.0W,Peabody School Creek Mining Llc,2016-02-08,,,,,,Miscellaneous - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
4,wyDG1000,WYwr_M1,WYwr_V1,WYwr_O1,,,,,wadeID1,Groundwater,,,,4326,,,,42.45147,-104.21279,,,POD,Schneider No 6 Well,wadeID5,,Well,WY,,,,,,,,,,700.00000,Fully Adjudicated,CR UW05/085,"Schneider, Henry And Edna",1976-05-20,,,,,,Irrigation - Ground Water,,,,,,0,,,,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491671,wyU99995,WYwr_M1,WYwr_V1,WYwr_O1,,,,North Platte River,wadeID2590,,,,,4326,,,,42.83176,-106.36906,,,POU,Midwest Refining Company Pipeline No 2 Act Amo...,wadeID10741,,,WY,,,,,,,,,,,Fully Adjudicated,CR CC97/058,City Of Casper,1917-10-30,,,,,,Municipal - Surface Water,,,,,,0,,,0.00000,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
491672,wyU99996,WYwr_M1,WYwr_V1,WYwr_O1,,,,North Platte River,wadeID2590,,,,,4326,,,,42.83176,-106.36906,,,POU,Midwest Refining Company Pipeline No 2 Act Amo...,wadeID10741,,,WY,,,,,,,,,,,Fully Adjudicated,CR CC97/058,City Of Casper,1917-10-30,,,,,,Municipal - Surface Water,,,,,,0,,,0.00000,,,,,,,http://seoweb.wyo.gov/e-Permit/Common/Login.aspx
491673,wyU99997,WYwr_M1,WYwr_V1,WYwr_O1,,,,North Platte River,wadeID2590,,,,,4326,,,,42.83176,-106.36906,,,POU,Midwest Refining Company Pipeline No 2 Act Amo...,wadeID10741,,,WY,,,,,,,

In [47]:
# Export the output dataframe
outdf.to_csv('Pwr_wyMain.zip', compression=dict(method='zip', archive_name='Pwr_wyMain.csv'), index=False) # The output, save as a zip
#dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.